In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from requests import get
import re
import pandas as pd
import csv
from unidecode import unidecode
whitelist = set('µÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõöøùúûüýþÿ')
import time

# Read input file
file_to_open = "Urbanisation DOI list_edit.csv"

df = pd.read_csv(file_to_open)

# Create header for output file
list1=['metrics_page'
       , 'Title'
       , 'Keywords'
       , 'Author1'
       , 'Author2'
       , 'Author3'
       , 'Author4'
       , 'Author5'
       , 'Author6'
       , 'Author7'
       , 'Author8'
       , 'Author9'
       , 'Author10'
       , 'Corresponding_author'
       , 'Corresponding_author_email'
       , 'Number_of_views_downloads'
       , 'Number_of_citations'
       , 'Citation1'
       , 'Citation2'
       , 'Citation3'
       , 'Citation4'
       , 'Citation5'
       , 'Citation6'
       , 'Citation7'
       , 'Citation8'
       , 'Citation9'
       , 'Citation10'
       , 'Citation11'
       , 'Citation12'
       , 'Citation13'
       , 'Citation14'
       , 'Citation15'
       , 'Citation16'
       , 'Citation17'
       , 'Citation18'
       , 'Citation19'
       , 'Citation20'
       , 'Citation21'
       , 'Citation22'
       , 'Citation23'
       , 'Citation24'
       , 'Citation25'
       , 'Citation26'
       , 'Citation27'
       , 'Citation28'
       , 'Citation29'
       , 'Citation30'
       , 'Citation31'
       , 'Citation32'
       , 'Citation33'
       , 'Citation34'
       , 'Citation35'
       , 'Citation36'
       , 'Citation37'
       , 'Citation38'
       , 'Citation39'
       , 'Citation40'
       ]
with open("urbanisation_author_and_citation_detail.csv", "a", encoding="utf-8", newline='') as fp:
    wr = csv.writer(fp, dialect='excel')
    wr.writerow(list1)

    
# Process input file
for index, row in df.iterrows():

    title = ' '
    keywords = ' '
    corresp_author = ' '
    corrsp_email = ' '
    views = '0'
    citations = '0'
    citation_details = []
    
    print(row['DOI Link'])
    page=row['DOI Link']
    
    # Get landing page 

#    options = Options()
#    options.headless = True
#    driver = webdriver.Firefox(options=options,
    driver = webdriver.Firefox(
        executable_path=r'C:\Users\vlewis\AppData\Local\Temp\Temp1_geckodriver-v0.30.0-win64.zip\geckodriver.exe')

    driver.get(page)

    content = driver.page_source
    soup = BeautifulSoup(content, "lxml")    
    
    # Get keywords
    keywords = soup.find("meta", attrs={"name": "keywords"})
    
    if keywords:
        keywords = keywords["content"]
    print(keywords)

    driver.close()
    
    # get metrics page
    page=page.replace("https://doi.org","https://journals.sagepub.com/doi/metrics")
    print(page)
    
#    options = Options()
#    options.headless = True
#    driver = webdriver.Firefox(options=options,
    driver = webdriver.Firefox(
        executable_path=r'C:\Users\vlewis\AppData\Local\Temp\Temp1_geckodriver-v0.30.0-win64.zip\geckodriver.exe')

    driver.get(page)

    content = driver.page_source
    soup = BeautifulSoup(content, "lxml")   
    
    # title
    title = soup.find("div", attrs={"class": "publicationContentTitle"})
    # print(title)
    if title:
        title = title.text.replace('\n', ' ').strip()
        title = ''.join(c if c in whitelist else unidecode(c) for c in title)
    print(title)
    
    # author
    dir_author_name = soup.find("div", attrs={"class": "publicationContentAuthors"})
    author_name = [None] * 10
    artTags = dir_author_name.find_all("div", {"class": "authorLayer"})
    i = 0
    for tag11 in artTags:
        tag11 = tag11.text.replace('\n', ' ').strip()
        tag11 = tag11.replace(u'\xa0', u' ')
        tag11 = ''.join(c if c in whitelist else unidecode(c) for c in tag11)
        author_name[i] = tag11.replace('See all articles by this author Search Google Scholar  for this author', ' ')
        i = i + 1
    print(author_name)
    
    # corresponding author
    corresp_author = soup.find("div", attrs={"class": "artice-notes"})
    if corresp_author:
        corresp_author = corresp_author.text.replace('Corresponding Author:', ' ').strip()
        corresp_author = corresp_author.replace('Corresponding author:', ' ').strip()
        corresp_author = corresp_author.split("E-mail",1)[0]
        corresp_author = ''.join(c if c in whitelist else unidecode(c) for c in corresp_author)
        print(corresp_author)
    
    # corresponding author email  
    corrsp_email = ' '
    def decodeEmail(e):
        de = ""
        k = int(e[:2], 16)

        for i in range(2, len(e)-1, 2):
            de += chr(int(e[i:i+2], 16)^k)
        return de
    
    prot_email = soup.find("span", {"class": "__cf_email__"})
    if prot_email:
        e = prot_email["data-cfemail"]
        corrsp_email = decodeEmail(e)
        print(corrsp_email)
        
    # number of views
    views = soup.find("div", attrs={"class": "widget literatumContentItemDownloadCount none articleUsage widget-none widget-compact-horizontal"})
    
    #dir_views
    views = views.text.replace('\n', ' ').strip().split(":",1)[1]
    print(views)
    
    # citations
    dir_citations = soup.find("div", attrs={"class": "widget literatumContentItemServiceCitaionLinks none widget-none widget-compact-all"})

    for table in dir_citations.find_all("table"):
        t_value = []
        for td in table.find_all("td"):
            t_value.append(td.text.replace('\n', ' ').strip())
    citations = t_value[1]
    print(citations)

    if citations != '0':
        print('found citations')
    
    driver.close()
    
    # access citations
    page=row['DOI Link']
    page=page.replace("https://doi.org","https://journals.sagepub.com/doi/citedby")
    print(page)
    
#    options = Options()
#    options.headless = True
#    driver = webdriver.Firefox(options=options,
    driver = webdriver.Firefox(
        executable_path=r'C:\Users\vlewis\AppData\Local\Temp\Temp1_geckodriver-v0.30.0-win64.zip\geckodriver.exe')

    driver.get(page)

    content = driver.page_source
    soup = BeautifulSoup(content, "lxml")   
    
    # citations details
    citation_details = [None] * 40
    i = 0
    if (citations != '0'and page != 'https://journals.sagepub.com/doi/citedby/10.1177/2455747119863891'):
        count = -1
        for tag in soup.find("div", attrs={"class": "citedBySection"}):
            tag1 = ''
            tag2 = ''
            tag3 = ''
            tag4 = ''
            tag5 = ''
            print(count)
    #    print('citedby')
            artTags = tag.find_all("div", {"class": "articleTitle"})
            for tag1 in artTags:
                print('articletitle')
                print(tag1.text)
                tag1 = tag1.text
            authTags = tag.find_all("div", attrs={"class": "authors"})
            for tag2 in authTags:
    #        print('authors')
                print(tag2.text)
                tag2 = tag2.text
            journalTags = tag.find_all("div", attrs={"class": "journalTitle"})
            for tag3 in journalTags:
    #        print('journaltitle')
                print(tag3.text)  
                tag3 = tag3.text
            pubdateTags = tag.find_all("div", attrs={"class": "pubDate"})
            for tag4 in pubdateTags:
    #        print('pubdate')
                print(tag4.text)   
                tag4 = tag4.text
            linkTags = tag.find_all("a", attrs={"class": "ref"})
            for tag5 in linkTags:
    #        print('link')
                print(tag5["href"])  
                tag5 = tag5["href"]
            count = count + 1
#            print(tag1)
            if count > 1:
                citation_tags = str(tag1) +'\n'+str(tag2)+'\n' +str(tag3)+'\n' +str(tag4)+'\n' +str(tag5)
#                citation_tags = str(tag1) +'\n'+str(tag2)+'\n' +str(tag3)+'\n' +str(tag4)+'\n' +"=HYPERLINK(" +'"'+ tag5+'"' +","+'"' + tag5+'"' +")"
                citation_tags = ''.join(c if c in whitelist else unidecode(c) for c in citation_tags)
                citation_details[i] = citation_tags    
                i = i + 1
#        
    driver.close()
    
#   write to output
    print('write to output')
    list1=["=HYPERLINK(" +'"'+ page+'"' +","+'"' + page+'"' +")"
       , title
       , keywords
       , author_name[0]
       , author_name[1]
       , author_name[2]
       , author_name[3]
       , author_name[4]
       , author_name[5]
       , author_name[6]
       , author_name[7]
       , author_name[8]
       , author_name[9]
       , corresp_author
       , corrsp_email
       , views
       , citations
       , citation_details[0]
       , citation_details[1]   
       , citation_details[2]   
       , citation_details[3]   
       , citation_details[4]   
       , citation_details[5]   
       , citation_details[6]   
       , citation_details[7]   
       , citation_details[8]   
       , citation_details[9]   
       , citation_details[10]
       , citation_details[11]   
       , citation_details[12]   
       , citation_details[13]   
       , citation_details[14]   
       , citation_details[15]   
       , citation_details[16]   
       , citation_details[17]   
       , citation_details[18]   
       , citation_details[19]  
       , citation_details[20]
       , citation_details[21]   
       , citation_details[22]   
       , citation_details[23]   
       , citation_details[24]   
       , citation_details[25]   
       , citation_details[26]   
       , citation_details[27]   
       , citation_details[28]   
       , citation_details[29]  
       , citation_details[30]
       , citation_details[31]   
       , citation_details[32]   
       , citation_details[33]   
       , citation_details[34]   
       , citation_details[35]   
       , citation_details[36]   
       , citation_details[37]   
       , citation_details[38]   
       , citation_details[39]  
          ]
    
    print(list1)
    with open("urbanisation_author_and_citation_detail.csv", "a", encoding="utf-8", newline='') as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(list1)
        
driver.quit()
      
################## End of Code ###############

https://doi.org/10.1177/24557471211021507


<ipython-input-8-9102d4738ed3>:100: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(


WebDriverException: Message: 'geckodriver.exe' executable needs to be in PATH. 
